<a href="https://colab.research.google.com/github/choarauc/form/blob/main/Reconnaissance_vocale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Visualisation et lecture d'un fichier audio
Concepts fondamentaux : Fréquence d'échantillonnage, Spectrogramme, échelle mel ...
Augmentation de données
Parallèle avec la reconnaissance de l'écriture manuscrite.
Fonction de coût : Connectionist Temporal Classification (CTC)`
Architecture du modèle
Décodeur : Best path decoding et Beam search decoder
Première évalation du modèle
Modèle de langage.

In [ ]:
import pandas as pd
df = pd.read_csv("libriSpeech.csv")
df.head()

Charger le premier fichier audio de df à l'aide de la fonction load_audio, et stocker le tableau de données et la fréquence d'échantillonnage sous le nom respectif audio et fe.

In [ ]:
import librosa
def load_audio(audio_path):
    return librosa.load(audio_path, sr=None)
audio, fe = load_audio(df.iloc[0,0])

Ecouter le fichier audio en utilisant la fonction Audio.

In [ ]:
from IPython.display import Audio
Audio(audio, rate=fe)

In [ ]:
#  transcription de ce fichier audio.

df.iloc[0,1]

Fréquence d'échantillonnage

Afficher le signal audio à l'aide de la fonction plot_audio.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def plot_audio(audio_data, fe):
    # Intervalle de temps entre deux points.
    dt= 1/fe 
    # Variable de temps en seconde.
    t = dt*np.arange(len(audio_data)) 
    plt.plot(t, audio_data)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")

plt.figure(figsize=(10,5))
plot_audio(audio, fe)

 observer les phénomènes d'intermodulation (fréquence d'échantillonnage trop faible).

 NB : Le Théorème de Shanon énonce qu'un échantillonnage à la fréquence d'échantillonnage  fefe  ne peut transmettre sans perte d'information que des fréquences inférieures à  Fe2Fe2 . Plus simplement, la fréquence d'échantillonnage  fefe  doit être au moins deux fois plus importante que la fréquence maximale du signal analogique.

In [ ]:
from ipywidgets import interactive
import seaborn as sns
sns.set()
def sampling(fe):
    duration = 1 # Durée du signal
    f = 3 # Fréquence du signal
    t_real = duration/10000*np.arange(10000) # On utilisera une fréquence d'échantillonnage très importante pour simuler le signal analogique
    real_fonction = np.sin(2*np.pi*f*t_real)
    plt.figure(figsize=(10,5))
    plt.plot(t_real, real_fonction, label='Signal Analogique')
    
    t_s = duration/(fe-1)*np.arange(fe)
    sampling_fonction = np.sin(2*np.pi*f*t_s)
    # Plot the black line
    for i in range(len(t_s)):
        plt.plot([t_s[i], t_s[i]], [0, sampling_fonction[i]], '-k')
        
    plt.plot([-0, 1], [0, 0], '-k', alpha=0.5, linewidth=0.5)
    plt.plot(t_s, sampling_fonction,'r-o', label='Signal Numérique')
    plt.xlabel("Time (s)")
    plt.xlim([-0.05, 1.05])
    plt.ylabel("Amplitude")
    plt.legend()
    
interactive_plot = interactive(sampling, fe=(2, 50))

interactive_plot

Représentation fréquentielle (ou spectrale)


variable  y=sin(2πfla⋅t)y=sin⁡(2πfla⋅t)  pour une fréquence d'échantillonnage de 16000 Hz et d'une durée de 2 seconde.

Écouter l'enregistrement y à l'aide de la fonction Audio.

In [ ]:
fe = 16000 # Hz
f_la = 440 # Hz
t = 1/fe*np.arange(2*fe) # Time
y = np.sin(2*np.pi*f_la*t) # Audio Amplitude
Audio(y, rate=fe)

 observer le signal temporel de la fonction ci-dessous et sa représentation fréquentielle.
f(t)=sin(2πf1t)+sin(2πf2t)
f(t)=sin⁡(2πf1t)+sin⁡(2πf2t)
 
   L'amplitude observée dans la figure suivante est  |s(n)||s(n)| .

In [ ]:
def sampling(f1, f2):
    duration = 0.05 # seconde
    fe = 8000
    dt = 1/fe
    t = dt*np.arange(fe*duration)
    y = np.sin(2*np.pi*f1*t) + np.sin(2*np.pi*f2*t)# sin(w*t)
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plot_audio(y, fe)
    plt.title("Temporal signal")
    plt.subplot(122)
    y_fourier = np.abs(np.fft.fft(y))[:len(y)//2]
    plt.bar(fe/(2*len(y_fourier))*np.arange(len(y_fourier)),y_fourier, edgecolor='b')
    plt.title("Frequency signal")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    
interactive_plot = interactive(sampling, f1=(100, 2000,100), f2=(100, 2000,100))
interactive_plot

Représentation tridimensionnelle : Spectrogramme. 

In [ ]:
#  fonction plot_spectrogram, afficher le spectrogramme de notre fichier audio avec une fréquence d'échantillonnage fe.

import librosa
import seaborn as sns

def spectrogram(audio, fe, dt):
    return np.abs(librosa.stft(audio,
                        n_fft = int(dt*fe),
                        hop_length = int(dt*fe/2)
                        )
                 )

def plot_spectrogram(audio, fe, dt=0.02):
    im = spectrogram(audio, fe, dt)
    sns.heatmap(np.rot90(im.T), cmap='inferno', vmin=0, vmax=np.max(im)/3)
    loc, labels = plt.xticks()
    l = np.round((loc-loc.min())*len(audio)/fe/loc.max(), 2)
    plt.xticks(loc, l)
    loc, labels = plt.yticks()
    l = np.array(loc[::-1]*fe/2/loc.max(), dtype=int)
    plt.yticks(loc, l)
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")

plt.figure(figsize=(15,5))
plot_spectrogram(audio, fe)

# observer le signal temporel d'équation ci-dessous et son spectrogramme.
f(t)=tsin(2πf1t)+t2sin(2πf2t)
f(t)=tsin⁡(2πf1t)+t2sin⁡(2πf2t)

In [ ]:
def sampling(f1, f2, dt):
    fe = 16000
    duration = 0.25 #seconde
    t = np.arange(fe*duration)/fe
    y = t*np.sin(2*np.pi*f1*t) + t**2*np.sin(2*np.pi*f2*t)# sin(w*t)
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plot_audio(y, fe)
    for i in range(int(duration//dt)+1):
        plt.axvline(x=i*dt, color='r', linestyle='-')
    plt.subplot(122)
    plot_spectrogram(y, fe, dt=dt)
    
interactive_plot = interactive(sampling, f1=(100, 2000, 100), f2=(100, 500, 100), dt=(0.02,0.10,0.01))

interactive_plot

observer le signal temporel de notre fichier audio et son spectrogramme.

In [ ]:
def sampling(dt):
    duration = len(audio)/fe
    t = np.arange(fe*duration)/fe
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plot_audio(audio, fe)
    for i in range(int(duration//dt)+1):
        plt.axvline(x=i*dt, color='r', linestyle='-')
    plt.subplot(122)
    plot_spectrogram(audio, fe, dt=dt)
    
interactive_plot = interactive(sampling, dt=(0.01,0.10,0.01))

interactive_plot

LogMelSpectrogram
La perception auditive n'est pas la même pour une variation fréquentielle dans les basses fréquences par rapport aux hautes fréquences. L'échelle de Mel est une échelle de fréquences conçue pour qu'une variation constante dans le domaine des mels soit perçue comme une variation constante de fréquence (en Hz) par les auditeurs.

Prenons par exemple une fréquence de base de 1000Hz. Un son à 2000Hz ne sera pas perçu par un auditeur comme étant 2 fois supérieur à la fréquence de base.

Par contre, si on effectue le changement fréquentiel dans le domaine des mels, 1000Hz correspondent à 1000mels. Un son à 2000 mels, c'est-à-dire à 3428Hz, sera perçu comme étant deux fois plus aigu qu'un son à 1000Hz.




afficher le log du spectrogramme dans l'échelle mel.

In [ ]:
params = {
    'n_fft': 160*16,
    'frame_step': 160*8,
    'lower_edge_hertz': 0,
    'upper_edge_hertz': 8000,
    'num_mel_bins': 160
}

def logMelSpectrogram(audio, params, fe):

    stfts = librosa.stft(audio,
                        n_fft = int(params['n_fft']),
                        hop_length = int(params["frame_step"]),
                        center = False
                        ).T
    power_spectrograms = np.real(stfts * np.conj(stfts))


    linear_to_mel_weight_matrix = librosa.filters.mel(
                                sr=fe,
                                n_fft=int(params['n_fft']) + 1,
                                n_mels=params['num_mel_bins'],
                                fmin=params['lower_edge_hertz'],
                                fmax=params['upper_edge_hertz']
                    ).T

    mel_spectrograms = np.tensordot(
                power_spectrograms,
                linear_to_mel_weight_matrix,
                1
            )

    return (np.log(mel_spectrograms + 1e-8).astype(np.float16))


def plot_logMelSpectrogram(audio, params, fe):
    sns.heatmap(np.rot90(logMelSpectrogram(audio, params, fe)), cmap='inferno', vmin = -6)
    loc, labels = plt.xticks()
    l = np.round((loc-loc.min())*len(audio)/fe/loc.max(), 2)
    plt.xticks(loc, l)
    plt.yticks([])
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Mel)")
    
    
plt.figure(figsize=(20,5))
plot_logMelSpectrogram(audio, params, fe)

Influence du bruit sur un signal
Le bruit est un son indésirable perçu comme un grésillement. Il n'apporte aucune information supplémentaire et sa présence peut impacter la performance des systèmes de traitement de signal. Par ailleurs, la forme du bruit d'acquisition n'est pas la même selon récepteur.

En général, le signal indésirable est modélisé par un bruit blanc (distribution gaussienne) :
X∼(μ,σ2)
X∼N(μ,σ2)
 
P(x)=1σ2π⎯⎯⎯⎯√e−(x−μ)2/2σ2
P(x)=1σ2πe−(x−μ)2/2σ2
 

In [ ]:
# Définir un vecteur aléatoire de 5000 valeurs suivant la loi  X∼(0,1)X∼N(0,1)  à l'aide de la fonction normal de numpy.random.

# Afficher la courbe du vecteur aléatoire à l'aide de la fonction plot_audio et on choisira une fréquence d'échantillonnage de 8000.

# Afficher la densité du vecteur aléatoire.

In [ ]:
noise_audio = np.random.normal(0,1,5000) # noise signal
plt.figure(figsize=(12,5))
plt.subplot(121)
plt.title('Audio Signal')
plot_audio(noise_audio, fe) # plot noise signal
plt.subplot(122)
plt.title('Density')
plt.xlabel('Amplitude')
plt.ylabel('Density probabilité')
sns.distplot(noise_audio) # plot distribution of signal
plt.show();

afficher l'influence du bruit sur le signal et le spectrogramme.

In [ ]:
def random_noise(audio, rate):
    noises = np.random.normal(0,np.max(np.abs(audio))*rate, len(audio))
    return np.array(audio + noises)

# Audio file 0
audio, fe = load_audio(df.audio_path[0])
audio_noise = random_noise(audio, rate=0.05)

plt.figure(figsize=(15,15))
plt.subplot(2, 2, 1)
plt.title("Audio")
plot_audio(audio, fe)
plt.subplot(2, 2, 2)
plt.title("Spectre audio")
plot_logMelSpectrogram(audio, params, fe)
plt.subplot(2, 2, 3)
plt.title("Audio with noise")
plot_audio(audio_noise, fe)
plt.subplot(2, 2, 4)
plt.title("Spectre with noise")
plot_logMelSpectrogram(audio_noise, params, fe)
plt.show();

écouter le signal bruité.

In [ ]:
print("Noise signal")
Audio(audio_noise, rate=fe)
# print("Real signal")
# Audio(audio, rate=fe)

Augmentation de données

Pour eviter le sur-apprentissage 

Étirement du temps à l'aide de la fonction time_stretch de librosa.effects.

Décalage temporel aléatoire à l'aide de la fonction pad de numpy.

Appliquer des bruits en superposant nos fichiers audio avec des bruits enregistrés (ou à un bruit blanc).